In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm
from torch.autograd import Variable

In [5]:
MODEL_PATH = './data/'

In [ ]:
class PressureEncorder(nn.Module):
    def __init__(self, image_size = 41, patch_size = 4, num_channels = 40, encoder_stride = 4):
        super(PressureEncorder, self).__init__()
        config = ViTConfig(image_size = 41, patch_size = 4, num_channels = 40, encoder_stride = 4)
        self.hidden_size = int((image_size // encoder_stride)**2 + 1) * config.hidden_size
        self.ViT = ViTModel(config)
        
        self.linear = nn.Linear(self.hidden_size + 10, 10)
        
    def forward(self, x):
        pressure, surge = x
        hidden = self.ViT(pressure).last_hidden_state.reshape(-1, self.hidden_size)
        x = torch.concat([hidden, surge], dim = 1)
        x = self.linear(x)
        return x

In [2]:
# num classes : numer of output parameters
class LSTMSurge(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, more_hidden_size=10, final_values=10):
        super(LSTMSurge, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size,
            num_layers=num_layers, 
            batch_first=True
        )

        layers = [
            nn.Linear(hidden_size, more_hidden_size), 
            nn.GELU(), 
            nn.Linear(more_hidden_size, final_values)
        ]
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        h_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        c_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.mlp(h_out)
        
        return out

In [ ]:
class TransformerSurge(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, more_hidden_size=10, final_values=10, pre_trained=True):
        super(TransformerSurge, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.init_weights()

        if pre_trained:
            a = 0 # load
        
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size,
            num_layers=num_layers, 
            batch_first=True
        )

        layers = [
            nn.Linear(hidden_size, more_hidden_size), 
            nn.GELU(), 
            nn.Linear(more_hidden_size, final_values)
        ]
        self.mlp = nn.Sequential(*layers)

    def forward(self, x):
        h_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        c_0 = Variable(
            torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        )
        
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.mlp(h_out)
        
        return out

In [4]:
w = torch.linspace(1, 0.1, 10)[np.newaxis]

def custom_weighted_losses(output, target):
    loss = torch.mean(w * (output - target)**2)
    return loss

## Data Loading

Here the data is loaded and scaled

In [15]:
X_train = np.load('./data/X_train_surge_new.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge_new.npz')

# train
slp_train = X_train['slp']
t_slp_train = X_train['t_slp']

t_surge1_input_train = X_train['t_surge1_input']
t_surge2_input_train = X_train['t_surge2_input']

surge1_input_train = X_train['surge1_input']
surge2_input_train = X_train['surge2_input']

mean_surge1_input_train = np.mean(surge1_input_train, axis=1)
std_surge1_input_train = np.std(surge1_input_train, axis=1)
mean_surge2_input_train = np.mean(surge2_input_train, axis=1)
std_surge2_input_train = np.std(surge2_input_train, axis=1)

scaled_surge1_input_train = (surge1_input_train - mean_surge1_input_train[:,None]) / std_surge1_input_train[:,None]
scaled_surge2_input_train = (surge2_input_train - mean_surge2_input_train[:,None]) / std_surge2_input_train[:,None]

t_surge1_output_train = X_train['t_surge1_output']
t_surge2_output_train = X_train['t_surge2_output']

# test
slp_test = X_test['slp']
t_slp_test = X_test['t_slp']

t_surge1_input_test = X_test['t_surge1_input']
t_surge2_input_test = X_test['t_surge2_input']

surge1_input_test = X_test['surge1_input']
surge2_input_test = X_test['surge2_input']

mean_surge1_input_test = np.mean(surge1_input_test, axis=1)
std_surge1_input_test = np.std(surge1_input_test, axis=1)
mean_surge2_input_test = np.mean(surge2_input_test, axis=1)
std_surge2_input_test = np.std(surge2_input_test, axis=1)

scaled_surge1_input_test = (surge1_input_test - mean_surge1_input_test[:,None]) / std_surge1_input_test[:,None]
scaled_surge2_input_test = (surge2_input_test - mean_surge2_input_test[:,None]) / std_surge2_input_test[:,None]

t_surge1_output_test = X_test['t_surge1_output']
t_surge2_output_test = X_test['t_surge2_output']

Code to have a series of pressures the same as the surges

In [26]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [27]:
pressures_same_time_1 = np.empty((*(t_surge1_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge1_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_1[i, j, :, :] = slp_train[i, idx, :, :]

pressures_same_time_2 = np.empty((*(t_surge2_input_train.shape), 41, 41))
for i, time_series in enumerate(t_surge2_input_train):
    for j, time in enumerate(time_series):
        idx = find_nearest(t_slp_train[i,:].flatten(), time)
        pressures_same_time_2[i, j, :, :] = slp_train[i, idx, :, :]

In [ ]:
trainlen = int(0.9*datalen)
vallen = datalen - trainlen
train_idx, val_idx = torch.utils.data.random_split(np.arange(datalen), [trainlen, vallen])

train_data = list(zip(X_pressure_train, X_surge_train, Y_train))
val_data = list(zip(X_pressure_val, X_surge_val, Y_val))

batch_size = 8

train_dataloader = DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

val_dataloader = DataLoader(
    val_data,
    batch_size=batch_size,
    shuffle=False
)

## Data Training

In [3]:
num_epochs = 200
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

Training of the transformer to compute the features from the pressure

In [ ]:
epochs = 10

for epoch in range(epochs):
    model.train()
    for x1, x2, y in tqdm(train_dataloader, total = len(train_dataloader), leave=False):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model((x1, x2))
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for x1, x2, y in tqdm(val_dataloader, total = len(val_dataloader), leave = False):
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)
            pred = model((x1, x2))
            loss = criterion(pred, y)
            val_loss += loss.item()
    val_loss /= (len(val_loader)*batch_size)
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

In [ ]:
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))

In [ ]:
torch.save(model.state_dict(), MODEL_PATH + 'model.')